In [15]:
from stdlib_list import stdlib_list
libraries = stdlib_list("2.7") + stdlib_list("3.6")
lines = list(set(libraries))
lines = lines +['mimetypes','opcode', 'xmlrpc', 'pip', 'antigravity', 'encodings', 'pkg_resources','concurrent', 'cprofile', 'django ', 'restframework ', 'flask ', 'flask_restful ', 'guardian ', 'logging ', 'array ', 'binascii ', 'email ', 'parser ', 'audioop ', 'encodings ', 'ctypes ', 'concurrent ', 'builtins ', 'importlib ', 'faulthandler ', 'atexit ', 'math ', 'marshal ', 'unicodedata ', 'syslog ', 'zipimport ', 'pkg_resources ', 'cprofile ', 'sunaudiodev ', 'itertools ', 'mmap ', 'time ', 'ossaudiodev ', 'resource ', 'pip ', 'alembic ', 'bbabel ', 'click ', 'django ', 'flask ', 'lxml ', 'matplotlib ', 'networkx ', 'psutil ', 'pytest ', 'requests ', 'selenium ', 'simplejson ', 'six ', 'sqlalchemy ', 'tornado ', 'twisted ', 'alembic ', 'babel ', 'click ', 'Django ', 'guardian ', 'restframework ', 'flask ', 'flask_restful ', 'lxml ', 'matplotlib ', 'mockredis ', 'networkx ', 'psutil ', 'jwt ', 'pymongo ', 'pytest ', 'dateutil ', 'yaml ', 'requests ', 'selenium ', 'simplejson ', 'six ', 'gridfs ', 'bson ', 'antigravity ', 'opcode']
lines = list(set(lines))

In [16]:
import requests
def crossword():
    url = 'https://crossword.stxnext.com/api/pyconpl2017/challenges'
    cookie = {'session': '.eJwlj0lqAzEQAP-isw-tXqSWPzNMb8QYEpixTyF_90DORUHVb9vqyPOr3V_HO29te0S7t1FiRjAFigMwxz6B9hjpZnkBscBCVmBfQlJCNPYC9MEGq0om7ivWgBxMac7Wg4jBTSBZfbiGghHbpLCe2QEgFKWvmVnebs3Po7bXzzO_r55Swd2C5nKZKyYTc2eokbUwUBXEEXVd3vvM43-Ctf19AAZ0PuY.DHh1Rg.tEB3jT39VrjOlBSf91s_pxlxIcI'}
    r = requests.get(url, cookies=cookie)
    urls = r.content.decode().split("\n")
    url = ([url for url in urls if len(url) > 5]).pop()
    ID = url.split('/').pop().replace('"', '')

   

    url = 'https://crossword.stxnext.com/api/pyconpl2017/challenges/{}'.format(ID)
    cookie = {'session': '.eJwlj0lqAzEQAP-isw-tXqSWPzNMb8QYEpixTyF_90DORUHVb9vqyPOr3V_HO29te0S7t1FiRjAFigMwxz6B9hjpZnkBscBCVmBfQlJCNPYC9MEGq0om7ivWgBxMac7Wg4jBTSBZfbiGghHbpLCe2QEgFKWvmVnebs3Po7bXzzO_r55Swd2C5nKZKyYTc2eokbUwUBXEEXVd3vvM43-Ctf19AAZ0PuY.DHh1Rg.tEB3jT39VrjOlBSf91s_pxlxIcI'}
    r = requests.get(url, cookies=cookie)

    from scrapy.selector import Selector
    url_base = 'https://crossword.stxnext.com/pyconpl2017/challenges/{}'.format(ID)
    req = requests.get(url_base, cookies=cookie)
    hints = Selector(text=req.content.decode()).xpath('/html/body/section/div/div/div[1]/pre/code/text()')
    all_hints = lines
    if hints:
        hints = hints.extract()[0].split('\n')
        hints = [hint.replace('django-', '').replace('django', '').lower() for hint in hints]
        hints.append('gridfs')
        hints.append('dateutil')
        hints.append('mockredis')
        all_hints += hints
        all_hints = list(set(all_hints))


    data = r.content.decode().split("\n")

    def get_words(data, down=True):
        words = []
        word = ''
        rows = len(data)
        columns = len(data[0])


        if down:
            for i in range(0, columns):
                for j in range(0, rows):
                    if data[j][i] != ' ':
                        word += data[j][i]
                    elif data[j][i] == ' ':
                        if word:
                            if len(word) > 1:
                                words.append((word, (j-1, i), down))
                        word = ''
                if word:
                    if len(word) > 1:
                        words.append((word, (j, i), down))
                    word = ''
        else:
            for j in range(0, rows):
                for i in range(0, columns):
                    if data[j][i] != ' ':
                        word += data[j][i]
                    elif data[j][i] == ' ':
                        if word:
                            if len(word) > 1:
                                words.append((word, (j, i-1), down))
                        word = ''
                if word:
                    if len(word) > 1:
                        words.append((word, (j, i), down))
                    word = ''
        return words

    words = get_words(data, False)
    words += get_words(data, True)

    import re
    def find_matches(words):
        result = []
        words.sort(key = lambda s: len(s[0]))
        words.reverse()
        for word in words:
            matches = []
            for line in all_hints:
                if len(line) == len(word[0]):
                    match = re.match(word[0].replace('*', '.'), line)
                    if match:
                        matches.append(match[0])
            if not matches:
                print('no match for {}'.format(word[0]))
            word = (word[0], word[1], word[2], tuple(matches))
            #print (word)
            result.append(word)
        return result

    words = find_matches(words)

    def data_fill(data, word):
        import random
        j, i = word[1]
        down = word[2]
        match = word[3][random.randint(0, len(word[3])-1)]
        length = len(match)

        if down:
            j -= length - 1
            x = 0
            for j in range(j, j+length):
                data[j] = list(data[j])
                data[j][i] = match[x]
                data[j] = ''.join(data[j])
                x += 1
        else:
            i -= length - 1
            x = 0

            data[j] = list(data[j])
            for i in range(i, i+length):
                data[j][i] = match[x]
                x += 1
            data[j] = ''.join(data[j])
        return data

    def solve(data, words, rerun=False):
        to_try = []
        for word in words:
            if len(word[3]) == 1 or (len(word[3]) > 1 and rerun):
                data = data_fill(data, word)
        words = get_words(data, False) +  get_words(data, True)
        words = find_matches(words)
        if not rerun:
            data = solve(data, words, True)
        #print(*data, sep='\n')
        return data

    solved_data = data
    print(*solve(solved_data, words), sep='\n')
    res = requests.put(url, cookies=cookie, data='\n'.join(data))
    return res.content

In [17]:
from requests.exceptions import ConnectionError
hello = False
def run_x(x):
    for i in range(0, x):
        crossword()
    return True
while not hello:
    try:
        hello=run_x(1000)
    except ConnectionError:
        pass

         t                                                      
      copy                                                      
         p s                                                    
      sqlite3                                                   
     c   n c                                                    
    stringprep s                                                
     y     e   y                                                
     pickletools                                                
   t e     s   c                                                
  curses       o                                                
   r           n                                   m            
   t           f                                   o            
 filecmp     cgi                                   d            
   e o  p      g                                 f u            
     m  dbm                                      a l       j    
   poplib              o 

         t                                                      
      copy                                                      
         p s                                                    
      sqlite3                                                   
     c   n c                                                    
    stringprep s                                                
     y     e   y                                                
     pickletools                                                
   t e     s   c                                                
  curses       o                                                
   r           n                                   m            
   t           f                                   o            
 filecmp     cgi                                   d            
   e o  p      g                                 f u            
     m  dbm                                      a l       j    
   poplib              o 

         t                                                      
      copy                                                      
         p s                                                    
      sqlite3                                                   
     c   n c                                                    
    stringprep s                                                
     y     e   y                                                
     pickletools                                                
   t e     s   c                                                
  curses       o                                                
   r           n                                   m            
   t           f                                   o            
 filecmp     cgi                                   d            
   e o  p      g                                 f u            
     m  dbm                                      a l       j    
   poplib              o 

         t                                                      
      copy                                                      
         p s                                                    
      sqlite3                                                   
     c   n c                                                    
    stringprep s                                                
     y     e   y                                                
     pickletools                                                
   t e     s   c                                                
  curses       o                                                
   r           n                                   m            
   t           f                                   o            
 filecmp     cgi                                   d            
   e o  p      g                                 f u            
     m  dbm                                      a l       j    
   poplib              o 

         t                                                      
      copy                                                      
         p s                                                    
      sqlite3                                                   
     c   n c                                                    
    stringprep s                                                
     y     e   y                                                
     pickletools                                                
   t e     s   c                                                
  curses       o                                                
   r           n                                   m            
   t           f                                   o            
 filecmp     cgi                                   d            
   e o  p      g                                 f u            
     m  dbm                                      a l       j    
   poplib              o 

...

                                                                  l                         
                                                           sysconfig                        
                                                              u   n                         
                                                         optparse e                         
                                                              s   c                         
                                                              e   a                         
                                                              s s c                         
                                                                o h                         
                                                               sched    d c                 
                                                                k       o m                 
                                                         cprofile     

                                                                  l                         
                                                           sysconfig                        
                                                              u   n                         
                                                         optparse e                         
                                                              s   c                         
                                                              e   a                         
                                                              s s c                         
                                                                o h                         
                                                               sched    d c                 
                                                                k       o m                 
                                                         cProfile     

                                                                  l                         
                                                           sysconfig                        
                                                              u   n                         
                                                         optparse e                         
                                                              s   c                         
                                                              e   a                         
                                                              s s c                         
                                                                o h                         
                                                               sched    d c                 
                                                                k       o m                 
                                                         cprofile     

                                t g                                      
                              turtle                                     
       c                        a o                                      
       o               q   w    c b                                      
itertools              u   a    e                                        
       o         c   b e   r enum                                        
       r  m    p o   i u   n    a                                        
       s  a    o n contextlib o l                                        
     keyword   p f   a     n  p l   t                                    
       s  s    l i   s s pkg_resources                                   
         threading socket  s  r c   l                                    
        m a    b p   i l      a     n                                    
       builtins  a   i e      t secrets                                  
        l     h  r t   c      o     t 

IndexError: string index out of range

In [19]:
ID

NameError: name 'ID' is not defined

['                                                 p                                   ',
 '                                                 k                                   ',
 '                                                 g                                   ',
 '                                                 _                                   ',
 '                                              xmlrpc                                 ',
 '                                                 e                                   ',
 '                                              *urses                                 ',
 '                                             *   o                                   ',
 '                                             su*au                                   ',
 '                                           c g   r                                   ',
 '                              *          s * i   c                                   ',
 '        